# Fourier transform in option pricing

In [1]:
import numpy as np
from scipy.integrate import quad

Let $\phi_T$ be the characteristic function of $\ln (S_T)$. We consider call price $C_T(k)$ of $(T, e^k)$. Then, 
$$C_T(k) = \frac{e^{-\alpha k}}{\pi} \int_0^\infty e^{-i w k} \psi_T(w) dw,$$
where
$$\psi_T(w) = \frac{e^{-rT} \phi_T(w - (\alpha + 1) i)}{\alpha^2 + \alpha - w^2 + i (2\alpha+1)w}.$$

In [2]:
class FTCall:
    def __init__(self, model):
        self.md = model
        
    #$\psi$ function for carr-madan method
    def psi(self, w, T, alpha):
        o1 = np.exp(-self.md.r*T)
        o1 = o1*self.md.charfun(T, w - (alpha+1)*1j)
        o2 = alpha**2+alpha-w**2+1j*(2*alpha+1.)*w
        return o1/o2
    
    #carr-madan method with damping
    def price_cm(self, K, T, alpha = 1.5):
        k = np.log(K)
        integrand = lambda w: (np.exp(-1j*w*k)*self.psi(w, T, alpha)).real
        integral = quad(integrand, 0, np.inf)[0]
        return np.exp(-alpha*k)/np.pi*integral

We denote by $GBM(s, r, \sigma^2)$ the dynamics
$$d S_t = r S_t dt + \sigma S_t dW_t, S_{0} = s$$
Non-negativity of the GBM process is good for modeling stock price, namely BSM.
Let
$X_t = \ln S_t$ be the log price. Then, the characteristic function of $X_t$ is
$$\phi_t = \exp \{iu (s + (r-\frac 1 2 \sigma^2)t) - \frac{u^2 \sigma^2 t}{2}\}.$$



In [2]:
class BSM:
    def __init__(self, S0, r, sigma):
        self.S0 = S0
        self.r = r
        self.sigma = sigma
        
    #characteristic function for $ln S_T$
    def charfun(self, T, u):
        o1 = np.log(self.S0) + (self.r - self.sigma**2/2.0)*T
        o2 = self.sigma**2*T/2.
        o3 = 1j*u*o1 - u**2*o2
        return np.exp(o3)

In [4]:
# #### bsm test    
bsm = BSM(100, 0.0475, 0.2)
ftc = FTCall(bsm)
ans = ftc.price_cm(110, 1., alpha = 1.5)
print(
      f'BSM FT price by carr-madan is \n >> {ans}'
      )

BSM FT price by carr-madan is 
 >> 5.94327318345284


In the Heston model, the dynamic involves two processes $(S_{t}, \nu_{t})$.
More precisely, the asset price $S$ follows generalized geometric Brownian motion with random volatility process $\sqrt{\nu_{t}}$, i.e.  
$$d S_{t} = r S_{t} dt + \sqrt{\nu_{t}} S_{t} dW_{1,t},$$
while squared of volatility process $\nu$ follows CIR process
$$ d \nu_{t} = \kappa (\theta - \nu_{t}) dt + \xi \sqrt{\nu_{t}} (\rho dW_{1,t} + 
\bar \rho d W_{2,t})$$
with $\rho^{2} + \bar \rho^{2} = 1.$ 

A Heston model with parameters
$(S_0, v_0, r, \kappa , \theta, \xi, \rho)$ has the characteristic function of $\ln S_T$ as
$$\phi_T (w) = f_1(w) f_2(w) f_3(w),$$
where
$$t_1 = \kappa - i \rho \xi u$$
$$D = \sqrt\{t_1^2 + (u^2 + iu) \xi^2,$$
$$G = (t_1 - D)/(t_1 +D),$$
$$t_2 = 1 - G e^{-DT}$$
$$f_1 = \exp( iu (\ln S_0 + rT))$$
$$f_2 = \exp(v_0 (1 - e^{-DT}) (t_1 - D)/ \xi^2 /t_2)$$
$$f_3 = \exp(\kappa \theta(T(t_1 -D) - 2 \ln (t_2/(1-G)))/\xi^2).$$
See page 53 of https://github.com/songqsh/songqsh.github.io/blob/master/paper/Ng05.pdf


In [22]:
# Heston model    
class Heston:
    def __init__(self, S0, v0, r, kappa, theta, xi, rho):
        self.S0 = S0
        self.v0 = v0
        self.r = r
        self.kappa = kappa
        self.theta = theta
        self.xi = xi
        self.rho = rho
        
    #char func for ln(S_T) from Ng05
    def charfun(self, T, u):
        t1 = self.kappa - self.rho*self.xi*1j*u
        D = np.sqrt(t1**2+(u**2+1j*u)*self.xi**2)
        G = (t1-D)/(t1+D)
        t2 = 1-G*np.exp(-D*T)
        f1 = np.exp(1j*u*(np.log(self.S0+self.r*T)))
        f2 = np.exp(self.v0*(1-np.exp(-D*T))*(t1-D)/self.xi**2/t2)
        f3 = np.exp(self.kappa*self.theta*(
            T*(t1-D)-2*np.log(t2/(1-G)))/self.xi**2)
        return f1*f2*f3

In [23]:
#test Heston
hes = Heston(100, 0.0175, 0., 1.5768, 0.0398, 0.5751, -0.5751)
ftc = FTCall(hes)
ans = ftc.price_cm(80, 10, alpha = 1.5)
print(
      f'Heston FT price by carr-madan is \n  >> {ans}'
      )
print('(ref. P61 of [Hirsa13])')

Heston FT price by carr-madan is 
  >> 32.57957785990924
(ref. P61 of [Hirsa13])


In [24]:
[ftc.price_cm(x, 10, alpha = 1.5) for x in [80, 100, 120]]

[32.57957785990924, 22.310218302545614, 14.78918741765821]